In [36]:
from datasets import load_dataset, Dataset, load_metric
import numpy as np
import nltk
from transformers import AutoTokenizer, T5TokenizerFast, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
import torch
import pandas as pd
from transformers.optimization import Adafactor, AdafactorSchedule

# Вариант 1. Самостоятельный fine-tune модели

In [2]:
gazeta = load_dataset("IlyaGusev/gazeta", revision="v2.0")

In [45]:
# кодируем данные для обработки моделью
def encode_data(input_sequences):
  task_prefix = "summarize: "
  if type(input_sequences) != list: 
    input_sequences = [input_sequences]
  encoded = tokenizer(
    [task_prefix + sequence for sequence in input_sequences],
    padding="longest",
    max_length=max_input,
    truncation=True,
    return_tensors="pt",)
  return encoded

In [40]:
#токенизируем датасет
def encode_dataset(dataset, max_output = 64):
    INPUT_IDS = []
    ATTENTION_MASK = []
    LABELS = []
    for i in range(len(dataset)):
        encoded_row = encode_data(dataset[i]['text'])
        input_ids, attention_mask = encoded_row.input_ids, encoded_row.attention_mask
        target_encoding = tokenizer(dataset[i]['summary'], padding="longest", max_length=max_output, truncation=True)
        labels = target_encoding.input_ids
        labels = torch.tensor(labels)
        labels[labels == tokenizer.pad_token_id] = -100
        INPUT_IDS.append(input_ids)
        ATTENTION_MASK.append(ATTENTION_MASK)
        LABELS.append(labels)
    data = Dataset.from_pandas(pd.DataFrame({'input_ids': list(np.array(INPUT_IDS)), 'attention_mask': list(np.array(ATTENTION_MASK)), 'labels': list(np.array(LABELS))}))
    return data

In [50]:
len(gazeta['train'])

60964

In [ ]:
encoded_gazeta = encode_dataset(gazeta['test'])

In [21]:
# альтернативный датасет
urukhan = load_dataset('UrukHan/t5-russian-summarization' )
urukhan_train = urukhan['train']
urukhan_test = urukhan['test'].train_test_split(0.02)['test'] 

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/452275 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23804 [00:00<?, ? examples/s]

In [4]:
metric = load_metric("rouge")
nltk.download('punkt')

C:\Users\Boris\AppData\Local\Temp\ipykernel_12644\2179309921.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Boris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
model_name = 'UrukHan/t5-russian-summarization' # Название модели из HuggingFace Hub
max_input = 1024  # Максимальная длинна входного текста (в токенах)
max_output  = 64  # Максимальная длинна сгенерированного краткого содержания (в токенах)
batch_size = 8 
output_dir = 'tmp_trainer'

In [11]:
tokenizer = T5TokenizerFast.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [12]:
model.config.max_length = max_output

In [13]:
train = gazeta['train']
test = gazeta['test']

In [14]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
  result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  result["gen_len"] = np.mean(prediction_lens)
  
  return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# ВНИМАНИЕ! НУЖНА CUDA. Можно использовать Google Collabs
# pip install transformers[torch]

# test = urukhan_test
# train = urukhan_train

training_args = Seq2SeqTrainingArguments(
  output_dir = output_dir,
  evaluation_strategy='steps',
  #learning_rate=2e-5,
  eval_steps=5000,
  save_steps=5000,
  num_train_epochs=1,
  predict_with_generate=True,
  per_device_train_batch_size=batch_size,
  per_device_eval_batch_size=batch_size,
  fp16=True,
  save_total_limit=2,
  #generation_max_length=256,
  #generation_num_beams=4,
  weight_decay=0.005,
  #logging_dir='logs',
)

# Оптимизатор
optimizer = Adafactor(
    model.parameters(),
    lr=1e-5,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False,
)
lr_scheduler = AdafactorSchedule(optimizer)

trainer = Seq2SeqTrainer(
  model=model,
  args=training_args,
  train_dataset = train,
  eval_dataset = test,
  optimizers = (optimizer, lr_scheduler),
  tokenizer = tokenizer,
  compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
predicts = model.generate(encode(input_sequences)) 

decoded = tokenizer.batch_decode(predicts, skip_special_tokens=True)

# Вариант 2. Просто исполь предобученную модель

In [ ]:
import torch
from transformers import GPT2Tokenizer, T5ForConditionalGeneration 
tokenizer = GPT2Tokenizer.from_pretrained('RussianNLP/FRED-T5-Summarizer',eos_token='</s>')
model = T5ForConditionalGeneration.from_pretrained('RussianNLP/FRED-T5-Summarizer')
device='cuda'
model.to(device)

input_text='<LM> Сократи текст.\n Текст (от лат. textus — ткань; сплетение, сочетание) — зафиксированная на каком-либо материальном носителе человеческая мысль; в общем плане связная и полная последовательность символов. Существуют две основные трактовки понятия «текст»: имманентная (расширенная, философски нагруженная) и репрезентативная (более частная). Имманентный подход подразумевает отношение к тексту как к автономной реальности, нацеленность на выявление его внутренней структуры. Репрезентативный — рассмотрение текста как особой формы представления информации о внешней тексту действительности.'
input_ids=torch.tensor([tokenizer.encode(input_text)]).to(device)
outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id,
                    num_beams=5,
                    min_new_tokens=17,
                    max_new_tokens=200,
                    do_sample=True,
                    no_repeat_ngram_size=4,
                    top_p=0.9)
print(tokenizer.decode(outputs[0][1:]))